In [10]:
import numpy as np
from PIL import Image
from io import BytesIO
import torch

from art.estimators.object_detection.pytorch_yolo import PyTorchYolo
from art.attacks.evasion import ProjectedGradientDescent

import cv2
import matplotlib
import matplotlib.pyplot as plt

from ultralytics import YOLO

from torchvision import datasets, transforms

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

# NOTE: This is a hack to get around "User-agent" limitations when downloading MNIST datasets
#       see, https://github.com/pytorch/vision/issues/3497 for more information
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [11]:
#convert onnx to pytorch
import onnx
from onnx2pytorch import ConvertModel

onnx_model = onnx.load('/home/aya/Desktop/Kitti-ObjectDetection/best.onnx')
pytorch_model = ConvertModel(onnx_model)

#put the model in evaluation mode
pytorch_model.eval()

ConvertModel(
  (Conv_/model.0/conv/Conv_output_0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (Sigmoid_/model.0/act/Sigmoid_output_0): Sigmoid()
  (Mul_/model.0/act/Mul_output_0): mul()
  (Conv_/model.1/conv/Conv_output_0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (Sigmoid_/model.1/act/Sigmoid_output_0): Sigmoid()
  (Mul_/model.1/act/Mul_output_0): mul()
  (Conv_/model.2/cv1/conv/Conv_output_0): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  (Sigmoid_/model.2/cv1/act/Sigmoid_output_0): Sigmoid()
  (Mul_/model.2/cv1/act/Mul_output_0): mul()
  (Constant_onnx::Split_140): Constant(constant=tensor([16, 16]))
  (Split_/model.2/Split_output_0): Split()
  (Conv_/model.2/m.0/cv1/conv/Conv_output_0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Sigmoid_/model.2/m.0/cv1/act/Sigmoid_output_0): Sigmoid()
  (Mul_/model.2/m.0/cv1/act/Mul_output_0): mul()
  (Conv_/model.2/m.0/cv2/conv/Conv_output_0): Conv2d(16, 16, kerne

In [12]:
#set variable device to cuda
device = 'cuda' if torch.cuda.is_available() else 'cpu'
pytorch_model.to(device)

ConvertModel(
  (Conv_/model.0/conv/Conv_output_0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (Sigmoid_/model.0/act/Sigmoid_output_0): Sigmoid()
  (Mul_/model.0/act/Mul_output_0): mul()
  (Conv_/model.1/conv/Conv_output_0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (Sigmoid_/model.1/act/Sigmoid_output_0): Sigmoid()
  (Mul_/model.1/act/Mul_output_0): mul()
  (Conv_/model.2/cv1/conv/Conv_output_0): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  (Sigmoid_/model.2/cv1/act/Sigmoid_output_0): Sigmoid()
  (Mul_/model.2/cv1/act/Mul_output_0): mul()
  (Constant_onnx::Split_140): Constant(constant=tensor([16, 16], device='cuda:0'))
  (Split_/model.2/Split_output_0): Split()
  (Conv_/model.2/m.0/cv1/conv/Conv_output_0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Sigmoid_/model.2/m.0/cv1/act/Sigmoid_output_0): Sigmoid()
  (Mul_/model.2/m.0/cv1/act/Mul_output_0): mul()
  (Conv_/model.2/m.0/cv2/conv/Conv_output_0): Con

In [13]:
import os
import numpy as np
from PIL import Image
import torch
from array import array

from torchvision import transforms

transform = transforms.Compose([transforms.PILToTensor()])

path="/home/aya/Desktop/Kitti-ObjectDetection/kitti-YOLOFormat/test/images/"

images=[]
for j in os.listdir(path):
    im = Image.open(path+j)
    im=im.resize((224,224))
    im=transform(im)
    images.append(im.type('torch.cuda.FloatTensor'))

path="/home/aya/Desktop/Kitti-ObjectDetection/kitti-YOLOFormat/test/labels/"

labels=[]
for k in os.listdir(path):
    y = np.loadtxt(path+k)
    y=np.resize(y,(20,5))
    # convert it to tensor
    y = torch.from_numpy(y)    
    labels.append(y)

In [14]:
#convert the list of images into tensor
images=torch.stack(images)
labels=torch.stack(labels)

In [15]:
import torchattacks
atk = torchattacks.PGD(pytorch_model, eps=8/255, alpha=2/255, steps=4)
# If, images are normalized:
# atk.set_normalization_used(mean=[...], std=[...])
adv_images = atk(images, labels)

NotImplementedError: Input with larger batch size than 1 not supported yet.